# Задание 4

В данной работе будет рассмотрено дифференциальное уравнение вида
 
$(p(x) y')' -q(x) y = -f(x)$, $0<x<l$

$y(0) = 0, y(l) = y$



In [626]:
import numpy as np
from scipy.integrate import quad

In [627]:
class DiffEquation:
    def __init__(self, p, q, f):
        self.p = p
        self.q = q
        self.f = f

In [628]:
def phi(j: int, x: float, xs: np.array) -> float:
    n = xs.size
    if j < 0 or j >= n:
        return 0
    elif j == 0:
        if x <= xs[1]:
            return (xs[1] - x) / (xs[1] - xs[0])
        else:
            return 0
    elif j == n - 1:
        if x >= xs[n - 1]:
            return (x - xs[n - 1]) / (xs[n - 1] - xs[n - 2])
        else:
            return 0
    else:
        if xs[j - 1] <= x <= xs[j]:
            return (x - xs[j - 1]) / (xs[j] - xs[j - 1])
        elif xs[j] <= x <= xs[j + 1]:
            return (xs[j + 1] - x) / (xs[j + 1] - xs[j])
        else:
            return 0
    

In [629]:
class SystemOfEquations:
    def __init__(self, de: DiffEquation, xs: np.array):
        n = xs.size
        self.d0 = np.zeros(n)
        self.d1 = np.zeros(n)
        self.d2 = np.zeros(n)
        self.v = np.zeros(n)
        self.d1[0] = 1
        self.d1[n - 1] = 1
        for j in range(1, n - 1):
            h1 = xs[j] - xs[j - 1]
            h2 = xs[j + 1] - xs[j]

            self.d0[j] = quad(lambda x: -de.p(x) + de.q(x) * (x - xs[j - 1]) * (xs[j] - x), xs[j - 1], xs[j])[0] / (h1 ** 2)
            self.d2[j] = quad(lambda x: -de.p(x) + de.q(x) * (x - xs[j]) * (xs[j + 1] - x), xs[j], xs[j + 1])[0] / (h2 ** 2)
            self.d1[j] = quad(lambda x: de.p(x) + de.q(x) * ((x - xs[j - 1]) ** 2), xs[j - 1], xs[j])[0] / (h1 ** 2) + quad(lambda x: de.p(x) + de.q(x) * ((xs[j + 1] - x) ** 2), xs[j], xs[j + 1])[0] / (h2 ** 2)
            self.v[j] = quad(lambda x: de.f(x) * phi(j, x, xs), xs[j - 1], xs[j + 1])[0]
            
    def solve(self):
        n = self.v.size

        for j in range(1, n):
            w = self.d0[j] / self.d1[j - 1]
            self.d1[j] -= w * self.d2[j - 1]
            self.v[j] -= w * self.v[j - 1]

        y = np.zeros(n)
        y[n - 1] = self.v[n - 1] / self.d1[n - 1]
        for j in range(n - 2, -1, -1):
            y[j] = (self.v[j] - self.d2[j] * y[j + 1]) / self.d1[j]
        return y


In [630]:
def approximate_func(x: float, xs: np.array, ys: np.array):
    n = xs.size
    h = xs[1] - xs[0]
    i = np.floor(x / h).astype(int)
    if i == n - 1:
        return ys[n - 1]
    else:
        return ys[i] * phi(i, x, xs) + ys[i + 1] * phi(i + 1, x, xs)

In [631]:
def compute_error(xs: np.array, ys: np.array, f):
    n = xs.size
    grid_size = n * 10
    grid = np.linspace(xs[0], xs[n - 1], grid_size)
    real = np.array([f(x) for x in grid])
    result = np.array([approximate_func(x, xs, ys) for x in grid])
    
    return np.sqrt(np.sum((real - result) ** 2))

# Эксперимент

Решим уравнение $y'' - y=-2 sin(x)$ на отрезке $[\pi]$ с граничными условиями $y(0) = y(\pi) = 0$.

Здесь $p(x) = 1, q(x) = 1, f(x) =  2sin(x)$

Ответом данного уравнения является функция $y(x) = sin(x)$

In [632]:
def solution(x):
    return np.sin(x)

In [633]:
def compute_bound(a, b, n, de):
    j_m = 0.15
    c = b ** 2 / 4 + 1
    dc = j_m * np.sqrt(c)
    h2 = ((b-a) / (n-1)) ** 2
    norm_f = quad(lambda x: de.f(x)**2, a, b)[0]
    return (c * dc)**2 * h2 * norm_f
    

In [634]:
a = 0
b = np.pi

for n in 10, 50, 100, 500, 1000:
    xs = np.linspace(a, b, n)
    diff_e = DiffEquation(p=lambda x: 1, q=lambda x: 1, f=lambda x: 2*np.sin(x))
    system = SystemOfEquations(diff_e, xs)
    ys = system.solve()
    error = compute_error(xs, ys, solution)
    bound = compute_bound(a, b, n, diff_e)
    print(f'При n = {n}, максимальная ошибка = {error}, при теоретической оценке сверху = {bound}')


При n = 10, максимальная ошибка = 0.048633532946589786, при теоретической оценке сверху = 0.7181073319791699
При n = 50, максимальная ошибка = 0.003630297137480767, при теоретической оценке сверху = 0.024226028275848712
При n = 100, максимальная ошибка = 0.0012582947353757367, при теоретической оценке сверху = 0.005934771338670828
При n = 500, максимальная ошибка = 0.00011077750642811121, при теоретической оценке сверху = 0.00023360024212879774
При n = 1000, максимальная ошибка = 3.908791049187804e-05, при теоретической оценке сверху = 5.828320201113301e-05


При различных размерах сетки наблюдаем погрешность зависящую квадратично от h, теоретическая оценка из книги  [Методы вычислений, Хакимзянов, Черный](http://www.ict.nsc.ru/matmod/files/textbooks/KhakimzyanovCherny-2.pdf) выполняется.